<a href="https://colab.research.google.com/github/pinballsurgeon/sweeping_analysis/blob/main/emergence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## installs

In [ ]:
!pip install pytrends
!pip install joypy

# install openai
!pip install openai

  Created wheel for pytrends: filename=pytrends-4.8.0-py3-none-any.whl size=16126 sha256=55f17ec3f97e1eac274a13c68303d973a53113eca8b72ac1b68339e5066de19e
  Stored in directory: /root/.cache/pip/wheels/07/6f/5c/8174f98dec1bfbc7d5da4092854afcbcff4b26c3d9b66b5183
Successfully built pytrends
     |████████████████████████████████| 40 kB 3.4 MB/s 
     |████████████████████████████████| 161 kB 5.5 MB/s 
  Created wheel for openai: filename=openai-0.15.0-py3-none-any.whl size=50093 sha256=af798e8894649ac8ea8f81045ede530606886d6cd9ef9e337fc5d8f804144096
  Stored in directory: /root/.cache/pip/wheels/bd/b1/b5/01a94056fd87ef0ed913b2fa6f1161076b730cf1449f579ab7
Successfully built openai


## imports

In [ ]:
import numpy
import pandas 
import re
import string
from difflib import SequenceMatcher

from pytrends.request import TrendReq
from pytrends         import dailydata
import pytrends
from operator         import index

import joypy
from matplotlib import cm
import matplotlib.pyplot as plt
import seaborn 

import openai

import networkx as nx

# supply openai api key via file 
openai.api_key = open('openai_key').read()


FileNotFoundError: ignored

## seed config

In [1]:
# connection strenth minimum boost, base on standard deviation
std_factor = .6


# seed topics
topics = ['asthma', 'bronchitis', 'sinusitis'
        , 'hypertension', 'pharmacy', 'primary care', 'bruise', 'sprain', 'strain', 'exczema', 'cellulitis', 'abscesses', 'strep throat'
        , 'ear infection', 'diabetes', 'migranes', 'pink eye', 'bladder infection', 'attention deficit', 'depression', 'anxiety']



## helper functions
  1. get_trend_frame(topics, years): returns a dataframe of topic trends
  2. stack_nodes_vertically(df_features):
  3. build_node_labels(df_flat):

In [ ]:
### 1. for a given list of topics, return dataframe of interests from pytrend
def get_trend_frame(topics, years):

  # initialize return frame
  df_total = pandas.DataFrame()

  # for each provided 
  for topic in topics:

    # pytrend request
    pytrend = TrendReq()
    pytrend.build_payload(kw_list=[topic]
                         ,timeframe=( 'today %s-y' % (years)))
    
    # load trend request
    df = pytrend.interest_over_time()

    # add column for new topic
    df_total[topic] = df[topic]

  # drop nans
  df_total = df_total.dropna()

  # return standard frame
  return df_total

In [ ]:
### 2. for each topic and interval of time, stack a vertical dataframe of value counts
def stack_nodes_vertically(df_features):

  # initialize buffers resources
  df_stk=pandas.DataFrame()
  globalst=[]

  # for each feature
  for col in df_features:                      # topic

      # for each time interval
      for val in df_features.index.weekofyear: # weeknum

          # time sensitive
          fs_val = df_features[col][val]       # full spectrum
          
          # iterate through each instance of max bin occurence, determining number of weeks written
          for x in range(0,int(abs(fs_val) * 100)):
          
              # build buffer list
              lst = [val, val, col]
              globalst.append(lst)

  # materialize vertically flattened
  df_flat = pandas.DataFrame(globalst
                            ,columns=['WeekNum'
                            ,'WeekNum_Rct'
                            ,'Flower_Tot'])
  
  return df_flat

In [ ]:
### 3. prefix ridgemap/joyplot labels with weeknum average
def build_node_labels(df_flat):
  
  # get all distinct nodes
  df_stats = df_flat.groupby('Flower_Tot').agg(['mean', 'count']).reset_index()

  # rename columns to node labels
  df_stats.columns = [' '.join(col).strip() for col in df_stats.columns.values]

  # sort by average time interval
  df_stats = df_stats.sort_values(by='WeekNum mean',ascending=True)

  # include average into node label
  df_stats['Name'] = ( ( df_stats['WeekNum mean'] * 100) + 1000).astype('int').astype('str') + ' '+ df_stats['Flower_Tot']

  # update flattened node space with averaged names
  for index, row in df_stats.iterrows():
      df_flat['Flower_Tot'] = df_flat['Flower_Tot'].replace(row['Flower_Tot'], row['Name'])

  return df_flat

In [ ]:
### 4. display ridgeplot of time frame, for 
def get_ridgeplot(df_flat):

  # configure and build ridgeplot
  fig, axes = joypy.joyplot(df_flat
                            , column=['WeekNum', 'WeekNum_Rct']
                            , by='Flower_Tot'
                            , overlap=1.2
                            , fill=True
                            , figsize=(22,12)
                            , x_range = [0,52]
                            , colormap=cm.cool                         
                            , ylim='own'
                          )

  # write ridgeplot
  plt.savefig('frank_jj.png')

  # plot
  plt.show()

In [ ]:
### 5. send openai question, receive response
def get_openai_response(question):

    # format open au request
    response = openai.Completion.create(
                      engine="text-davinci-001",
                      prompt=question,
                      temperature=.01,
                      max_tokens=250,
                      top_p=1,
                      frequency_penalty=50,
                      presence_penalty=0 )

    # parse and process open ai response
    response_choices = response["choices"]

    # replace blanks
    response = response_choices[0]["text"].strip('\n')

    return response

In [ ]:
def swap_tokens(topics, chk_topics, new_topics):

  buffer_lst = []


  # for each full list of topics
  for topic in topics:

    # for each graphed topics
    for chk_topic in chk_topics:

      # if 
      if topic == chk_topic:

        buffer_lst.append(topic)


  for new_topic in new_topics:
    buffer_lst.append(new_topic)


  return buffer_lst


In [ ]:
def get_pairwise_similarity(df_features, topics, std_factor):

  # pair wise correlation / similarity
  c = df_features.corr().abs()

  # flatten and sort
  so = c.unstack().sort_values(kind="quicksort")

  # remove low connections
  min_connection_strength = so.mean() + ( std_factor * so.std() )
  min_connection_cutoff = len(so) - len(so[so > min_connection_strength])

  # turn to frame
  df_slice = so[min_connection_cutoff:-len(topics)].to_frame()

  df_slice.reset_index(inplace=True)

  df_slice.columns = ['source','target','weight']

  return df_slice

  #df_slice.hist()

In [ ]:
def build_graph_network(df_slice):

  # build graph 
  G = nx.from_pandas_edgelist(df_slice
                            ,'source'          # parent
                            ,'target'          # child
                            , ['weight'])      # relationship


  # define plot area
  plt.figure(3,figsize=(12,12)) 


  # visualize graph 
  nx.draw(G                        
      ,with_labels=True
      ,edge_color='gray'
      ,node_color='white'
      ,font_size=22)
  
  return G

In [ ]:
def get_openai_generated_topics(topic):
  responses = []

  # retrieve 
  retrieval = get_openai_response('list objects related to %s' % (topic))
  responses.append(retrieval)

  new_tokens = re.split("; |, |\*|\n",responses[0].lower())

  return new_tokens

## gather, pre-process and stage graph data structure

In [ ]:
# request features
df_features = get_trend_frame(topics, '5')[topics]

# normalize feature space
df_features=(df_features-df_features.mean())/df_features.std()

# flatten feature space vertically
df_flat = stack_nodes_vertically(df_features)

# build node labels
df_flat = build_node_labels(df_flat)

# display ridgeplot of events
get_ridgeplot(df_flat)

In [ ]:
# time-based pairwise similiarity weights
df_slice = get_pairwise_similarity(df_features, topics, std_factor)

# display flattened histogram
df_slice.hist()

# build graph
G = build_graph_network(df_slice)

In [ ]:
### graph properties

# list nodes
G.nodes.items()

# number of edges in graph
G.number_of_edges()

# number of nodes in graph
G.number_of_nodes()



In [ ]:
lst_buffer = []
graph_topics = list(df_slice['source'].unique())

for graph_topic in graph_topics:
  topic_list = get_openai_generated_topics(graph_topic)

  for topic in topic_list:

    if len(topic) < 10 and len(topic) > 2:

      topic = topic.translate(string.punctuation)
      topic = re.sub(r'[^\w\s]','',topic)
      lst_buffer.append(topic)
  
  print(lst_buffer)



In [ ]:
lst_buffer.append('the fireworks')

In [ ]:
def reduce_similiar_topics(lst_ref, lst_new)):
  
  



  print('%s vs %s' % (lst_buffer[4], lst_buffer[8]))
  SequenceMatcher(None, lst_buffer[4], lst_buffer[8]).ratio()

In [ ]:
chk_topics = list(df_slice['source'].unique())

In [ ]:
swap_tokens(topics, chk_topics, new_tokens)